In [4]:
from dash import dash, Dash, callback, dcc, html
import dash_bootstrap_components as dbc
#import dash_core_components as dcc
#import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go

# Load data
clean_df = pd.read_csv('clean_df.csv')

vars_names = [var1 for var1 in clean_df.iloc[:,6:22].columns]
vars_currency =  clean_df.EURCZK #['CZK', 'EUR', 'GBP'] {#'EURCZK':'CZK','EURCZK':'EUR','EURGBP':'GBP'}
clean_df['year'] = pd.to_datetime(clean_df['date']).dt.year

# -------------------- Bar plot --------------------
bar_df = clean_df.iloc[:,5:22]
bar_cols = bar_df.columns[1:]

corr = [bar_df['PX'].corr(bar_df.iloc[:,1:][columns]) for columns in bar_cols]
corr = [round(num, 2) for num in corr]

fig_bar = go.Figure()

fig_bar.add_trace(
    go.Bar(
        y=bar_cols,  # Use 'y' for horizontal orientation
        x=corr,
        orientation='h',  # Set orientation to horizontal
        text=corr,
        textposition='outside'
    )
)
fig_bar.update_traces(marker_color='#3283FE', marker_line_color='rgb(0,0,0)', marker_line_width=1, opacity=0.8)
fig_bar.update_layout(width=250,height=320,autosize=False,margin=dict(l=30, r=10, t=10, b=10),paper_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(title='Coefficient of correlation',titlefont_size=12,tickfont_size=11, range=[0, 1.2],dtick=0.2,tickwidth=1),
    yaxis=dict(range=[-1, len(bar_cols)],tickfont_size=10,autorange="reversed"),
    template="plotly_white")

# -------------------- PX Index Line Plot --------------------
# Load dataset - main index DataFrame
mi_df = clean_df

# Initialize figure
fig_line = go.Figure()

# Add Traces
fig_line.add_trace(go.Scatter(x=list(mi_df.date),y=list(mi_df.high),name="High",line=dict(color="#1CBE4F")))
fig_line.add_trace(go.Scatter(x=list(mi_df.date),y=[mi_df.high.mean()] * len(mi_df),name="High Average",visible=False,line=dict(color="#1CBE4F", dash="dash")))   #33CFA5
fig_line.add_trace(go.Scatter(x=list(mi_df.date),y=list(mi_df.low),name="Low",line=dict(color="#C4451C")))
fig_line.add_trace(go.Scatter(x=list(mi_df.date),y=[mi_df.low.mean()] * len(mi_df.index),name="Low Average",visible=False,line=dict(color="#C4451C", dash="dash")))   #F06A6A

# Add Buttons and update plot and its area
fig_line.update_layout(
    updatemenus=[dict(active=0,buttons=list([
        dict(label="None",method="update",args=[{"visible": [True, False, True, False]},{"title": "PX Index","annotations": []}]),
        dict(label="High",method="update",args=[{"visible": [True, True, False, False]},{"title": "PX Index High","annotations": []}]),
        dict(label="Low",method="update",args=[{"visible": [False, False, True, True]},{"title": "PX Index Low","annotations": []}]),
        dict(label="High and Low",method="update",args=[{"visible": [True, True, True, True]},{"title": "PX Index","annotations": []}]),
    ]),direction="down",showactive=True,xanchor="left",yanchor="top",x=-0.02,y=1.2)],
    width=450,height=350,autosize=False,template="plotly_white",title_text="PX Index Time Series",legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1),
    xaxis_title='Year',yaxis_title='EUR')

# -------------------- Distribution plot --------------------
hist_df = [clean_df['PX']]
group_labels = ['PX Index'] 
colors = ['#3283FE']

fig_dist = ff.create_distplot(hist_df, group_labels, show_hist=False, colors=colors) 
fig_dist.update_layout(title_text='PX Index Distribution',xaxis_title = 'EUR',width=450,height=350,autosize=True,template="plotly_white", 
    legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1),margin=dict(l=5,r=5,b=10,t=10,pad=4))

#---------------- All Indices Line Plot ------------------------------
# Load data
all_line_df = clean_df.iloc[:,5:22]
date = clean_df['date']

#line_main_index_df = np.log(line_main_index_df)
#line_other_indeces_df = np.log(line_other_indeces_df)

fig_all_line = go.Figure()

# Main index
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['PX'], name='PX', line=dict(color='#325A9B', width=3))) # dash options include 'dash', 'dot', and 'dashdot'

# East European Indices - fifth and sixth wave - "BUX",'BET','BTE','CRE','WIG',
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['BUX'], name='BUX', line=dict(color='#3283FE', width=2, dash='dash'))) 
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['BET'], name='BET', line=dict(color='#3283FE', width=2, dash='dot'))) 
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['BTE'], name='BTE', line=dict(color='#3283FE', width=2, dash='dashdot'))) 
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['WIG'], name='WIG', line=dict(color='#3283FE', width=1))) 

# EU Founders + first wave of extensions - 'AEX','CAC','DAX','FTMIB','UKX',
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['AEX'], name='AEX', line=dict(color='#FEAF16', width=2, dash='dashdot'))) 
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['CAC'], name='CAC', line=dict(color='#FEAF16', width=2, dash='dash'))) 
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['DAX'], name='DAX', line=dict(color='#FEAF16', width=2, dash='dot'))) 
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['FTMIB'], name='FTMIB', line=dict(color='#FEAF16', width=2, dash='longdash'))) 
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['UKX'], name='UKX', line=dict(color='#FEAF16', width=2, dash='longdash'))) 

# Third and fourth waves of extensions - 'IBC','ATX','OMXS30',
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['IBC'], name='IBC', line=dict(color='#1CBE4F', width=2, dash='dash'))) 
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['ATX'], name='ATX', line=dict(color='#1CBE4F', width=2, dash='dot'))) 
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['OMXS30'], name='OMXS30', line=dict(color='#1CBE4F', width=2, dash='longdash'))) 

# Supranational indices - 'OMXN40','SX5E','SMI',
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['OMXN40'], name='OMXN40', line=dict(color='#C4451C', width=2, dash='longdash'))) 
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['SX5E'], name='SX5E', line=dict(color='#C4451C', width=2))) 
fig_all_line.add_trace(go.Scatter(x=date, y=all_line_df['SMI'], name='SMI', line=dict(color='#C4451C', width=2, dash='dashdot'))) 

fig_all_line.update_layout(title='All Indices Time Series',xaxis_title='Year',yaxis_title='EUR',width=780,height=350,template="plotly_white")
fig_all_line.update_layout(
    updatemenus=[dict(visible=True,type="buttons",  
         buttons=[
             dict(label="Linear",method="relayout",args=[{"yaxis.type": "linear"}]),
             dict(label="Log",method="relayout",args=[{"yaxis.type": "log"}]),],
             direction="right",showactive=True,xanchor="center",yanchor="top",y=1.3,x=0.9)]);  
# Add range slider
fig_all_line.update_layout(xaxis=dict(
    rangeselector=dict(
        buttons=list([
            dict(count=1,label="1m",step="month",stepmode="backward"),
            dict(count=6,label="6m",step="month",stepmode="backward"),
            dict(count=1,label="YTD",step="year",stepmode="todate"),
            dict(count=1,label="1y",step="year",stepmode="backward"),
            dict(step="all")])),
    rangeslider=dict(visible=True),type="date"))

# ------------------------------------------------

app = dash.Dash(external_stylesheets=[dbc.themes.FLATLY])

sidebar = html.Div(
    [
        dbc.Row(
            [
                html.H5('Settings'
                       # style={'margin-top': '8px', 'margin-left': '6px'}
                       )
                ],
            style={"height": "5vh"},
            className='bg-primary text-white font-italic',
            ),
        dbc.Row(
            [
                html.Div([
                    html.P('Currency',                       
                           style={'margin-top': '12px', 'margin-bottom': '4px'},
                           className='font-weight-bold'), 
                    dcc.RadioItems(id='currency-radio',
                                   options=[
                                       {'label': 'EUR', 'value': 'CZK_to_EUR'},
                                       {'label': 'CZK', 'value': 'EUR_to_CZK'}
                                   ],
                                    value='CZK_to_EUR',  # Default value
                                    labelStyle={'display': 'block'},  # Display radio items vertically
                                    style={"height": "15vh"}
                                  ), 
                    #dcc.Input(style={"margin-left": "15px"})
                    
                    html.P('Years',
                           #style={'margin-top': '10', 'margin-bottom': '10'},
                           style={'margin-bottom': '40'},
                           className='font-weight-bold'),
                    dcc.RangeSlider(id='year-range-slider',
                                    marks={
                                        year: {"label": str(year), "style": {"writing-mode": "vertical-rl"}} for year in range(clean_df['year'].min(), clean_df['year'].max() + 1)                        
                                    },                                   
                                    min=clean_df['year'].min(),
                                    max=clean_df['year'].max(),
                                    value=[clean_df['year'].min(), clean_df['year'].max()],
                                    step=1
                                   ), 
                    html.Hr()
                    ])
                ],
            style={'height': '40vh'}),
        dbc.Row(
            [
                html.Div([
                    html.P(id='bar_title', children='Correlation: PX and Other Indices', className='font-weight-bold'),# style={'margin-top': '2px'}),
                    dcc.Graph(id='bar-graph', 
                              figure=fig_bar, 
                              #style={'width': '100%', 'margin': 'auto'}
                             )
                    ]
                    #style={'width': '100%', 'margin-right': '4px'}# Set the width to 12 to make it take the full width of the sidebar
                    #style={'width': '100%', 'margin': 'auto'}  # Center the graph within the div
                )
            ],
            style={"height": "55vh"}#, 'margin': '2px'},
            )
        ]
    )

content = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Div([
                            #html.P(id='bar-title',
                            #       children='PX Index',
                            #       className='font-weight-bold'),
                            dcc.Graph(id="main-index-line-chart",
                                     figure=fig_line,
                                     className='bg-light',
                                     style={'width': '100%', 'height':'100', 'margin': 'auto'})
                        ])
                    ],
                    width=6
                    ), 
                dbc.Col(
                    [
                        #html.P('Distribution of Continuous Variable') fig_dist
                        dcc.Graph(id="dist-chart",
                                  figure=fig_dist,
                                  className='bg-light',
                                  style={'width': '100%', 'height':'100', 'margin': 'auto'})
                    ],
                    width=6,
                    style={"height": "50vh"}
                )
            ]),
        dbc.Row(
            [
                dbc.Col(
                    [
                        #html.P('Correlation Matrix Heatmap'),
                        dcc.Graph(id="all-line-chart",
                                figure=fig_all_line,
                                className='bg-light',
                                style={'width': '100%', 'height':'100', 'margin': 'auto'}),
                    ],
                    width=12
                    )
            ],
            style={"height": "50vh"}
        )
    ]
)

app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(sidebar, width=3, className='bg-light'),
                dbc.Col(content, width=9)
                ],
            style={"height": "100vh"}
            ),
        ],
    fluid=True
    )

# Callbacks bar chart
@app.callback(
    Output('bar-graph', 'figure'),
    Input('year-range-slider', 'value')
)

def update_bar_chart_output(selected_years):
    fig_bar_updated = go.Figure()
    
    # Filter clean_df based on selected years
    filtered_bar_df = clean_df[(clean_df['year'] >= selected_years[0]) & (clean_df['year'] <= selected_years[1])]
    filtered_bar_df = filtered_bar_df.iloc[:,5:22]
    filtered_bar_cols = filtered_bar_df.columns[1:]

    filtered_corr_values = [filtered_bar_df['PX'].corr(filtered_bar_df.iloc[:,1:][columns]) for columns in filtered_bar_cols]
    filtered_corr_values = [round(num, 2) for num in filtered_corr_values]

    fig_bar_updated.add_traces(go.Bar(y=filtered_bar_cols,x=filtered_corr_values,orientation='h',text=filtered_corr_values,textposition='outside'))
    fig_bar_updated.update_traces(marker_color='#3283FE', marker_line_color='rgb(0,0,0)', marker_line_width=1, opacity=0.8)
    fig_bar_updated.update_layout(width=250,height=320,autosize=False,margin=dict(l=30, r=10, t=10, b=10),paper_bgcolor='rgba(0,0,0,0)',template="plotly_white",
                xaxis=dict(title='Coefficient of correlation',titlefont_size=12,tickfont_size=11, range=[0, 1.2],dtick=0.2,tickwidth=1),
                yaxis=dict(range=[-1, len(filtered_bar_cols)],tickfont_size=10,autorange="reversed"))
    
    return fig_bar_updated


@app.callback(
    Output('main-index-line-chart', 'figure'),
    Input('year-range-slider', 'value'),
    Input('currency-radio', 'value')
)

def update_main_index_chart_output(selected_years, selected_currency):
    fig_line_updated = go.Figure()

    # Filter clean_df based on selected years
    filtered_main_line_df = clean_df[(clean_df['year'] >= selected_years[0]) & (clean_df['year'] <= selected_years[1])]
    exchange_rate_col = filtered_main_line_df['EURCZK']
    filtered_main_line_date_col = filtered_main_line_df['date']
    filtered_main_line_to_convert_df = filtered_main_line_df.iloc[:,2:6]

    if selected_currency == 'CZK_to_EUR':
        # Convert from CZK to EUR
        converted_values_df = pd.concat([filtered_main_line_date_col, filtered_main_line_to_convert_df], axis=1)
        currency = 'EUR'
    elif selected_currency == 'EUR_to_CZK':
        # Convert from EUR to CZK
        converted_values_df = filtered_main_line_to_convert_df.multiply(exchange_rate_col, axis=0)
        converted_values_df = pd.concat([filtered_main_line_date_col, converted_values_df], axis=1)
        currency = 'CZK'
    else:
        converted_values = []

    # Add Traces
    fig_line_updated.add_trace(go.Scatter(x=list(converted_values_df.date),y=list(converted_values_df.high),name="High",line=dict(color="#1CBE4F")))
    fig_line_updated.add_trace(go.Scatter(x=list(converted_values_df.date),y=[converted_values_df.high.mean()] * len(converted_values_df),
                                          name="High Average",visible=False,line=dict(color="#1CBE4F", dash="dash")))   #33CFA5
    fig_line_updated.add_trace(go.Scatter(x=list(converted_values_df.date),y=list(converted_values_df.low),name="Low",line=dict(color="#C4451C")))
    fig_line_updated.add_trace(go.Scatter(x=list(converted_values_df.date),y=[converted_values_df.low.mean()] * len(converted_values_df.index),
                                          name="Low Average",visible=False,line=dict(color="#C4451C", dash="dash")))   #F06A6A
    
    # Add Buttons and update plot and its area
    fig_line_updated.update_layout(
        updatemenus=[dict(active=0,buttons=list([
            dict(label="None",method="update",args=[{"visible": [True, False, True, False]},{"title": "PX Index","annotations": []}]),
            dict(label="High",method="update",args=[{"visible": [True, True, False, False]},{"title": "PX Index High","annotations": []}]),
            dict(label="Low",method="update",args=[{"visible": [False, False, True, True]},{"title": "PX Index Low","annotations": []}]),
            dict(label="High and Low",method="update",args=[{"visible": [True, True, True, True]},{"title": "PX Index","annotations": []}]),
            ]),direction="down",showactive=True,xanchor="left",yanchor="top",x=-0.02,y=1.2)],
        width=450,height=350,autosize=False,template="plotly_white",title_text="PX Index Time Series",legend=dict(orientation="h",
                                            yanchor="bottom",y=1.02,xanchor="right",x=1),
        xaxis_title='Year',yaxis_title=currency,)

    return fig_line_updated

@app.callback(
    Output('all-line-chart', 'figure'),
    Input('year-range-slider', 'value'),
    Input('currency-radio', 'value')
)
# Use the selected_years, and currency in your data processing
def update_all_line_plot(selected_years,selected_currency):
    fig_all_line_updated = go.Figure()

    filtered_all_line_df = clean_df[(clean_df['year'] >= selected_years[0]) & (clean_df['year'] <= selected_years[1])]
    exchange_rate_col = filtered_all_line_df['EURCZK']
    filtered_all_line_date_col = filtered_all_line_df['date']
    filtered_all_line_to_convert_df = filtered_all_line_df.iloc[:,5:22]

    if selected_currency == 'CZK_to_EUR':
        # Convert from CZK to EUR
        converted_values_all_df = pd.concat([filtered_all_line_date_col, filtered_all_line_to_convert_df], axis=1)
        currency = 'EUR'
    elif selected_currency == 'EUR_to_CZK':
        # Convert from EUR to CZK
        converted_values_all_df = filtered_all_line_to_convert_df.multiply(exchange_rate_col, axis=0)
        converted_values_all_df = pd.concat([filtered_all_line_date_col, converted_values_all_df], axis=1)
        currency = 'CZK'
    else:
        converted_values = []

    # Main index
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['PX'], name='PX', line=dict(color='#325A9B', width=3))) 
    # East European Indices - fifth and sixth wave - "BUX",'BET','BTE','CRE','WIG',
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['BUX'], name='BUX', line=dict(color='#3283FE', width=2, dash='dash'))) 
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['BET'], name='BET', line=dict(color='#3283FE', width=2, dash='dot'))) 
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['BTE'], name='BTE', line=dict(color='#3283FE', width=2, dash='dashdot'))) 
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['WIG'], name='WIG', line=dict(color='#3283FE', width=1)))    
    # EU Founders + first wave of extensions - 'AEX','CAC','DAX','FTMIB','UKX',
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['AEX'], name='AEX', line=dict(color='#FEAF16', width=2, dash='dashdot'))) 
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['CAC'], name='CAC', line=dict(color='#FEAF16', width=2, dash='dash'))) 
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['DAX'], name='DAX', line=dict(color='#FEAF16', width=2, dash='dot'))) 
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['FTMIB'], name='FTMIB', line=dict(color='#FEAF16', width=2, dash='longdash'))) 
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['UKX'], name='UKX', line=dict(color='#FEAF16', width=2, dash='longdash'))) 
    # Third and fourth waves of extensions - 'IBC','ATX','OMXS30',
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['IBC'], name='IBC', line=dict(color='#1CBE4F', width=2, dash='dash'))) 
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['ATX'], name='ATX', line=dict(color='#1CBE4F', width=2, dash='dot'))) 
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['OMXS30'], name='OMXS30', line=dict(color='#1CBE4F', width=2, dash='longdash'))) 
    # Supranational indices - 'OMXN40','SX5E','SMI',
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['OMXN40'], name='OMXN40', line=dict(color='#C4451C', width=2, dash='longdash'))) 
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['SX5E'], name='SX5E', line=dict(color='#C4451C', width=2))) 
    fig_all_line_updated.add_trace(go.Scatter(x=filtered_all_line_date_col, y=converted_values_all_df['SMI'], name='SMI', line=dict(color='#C4451C', width=2, dash='dashdot'))) 
    
    # Edit the layout
    fig_all_line_updated.update_layout(title='All Indices Time Series',xaxis_title='Year',yaxis_title=currency,width=800,height=350,template="plotly_white")
    fig_all_line_updated.update_layout(
        updatemenus=[dict(visible=True,type="buttons",  
             buttons=[
                dict(label="Linear",method="relayout",args=[{"yaxis.type": "linear"}]),
                dict(label="Log",method="relayout",args=[{"yaxis.type": "log"}]),],
                direction="right",showactive=True,xanchor="center",yanchor="top",y=1.3,x=0.9)]); 
    # Add range slider
    fig_all_line_updated.update_layout(xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,label="1m",step="month",stepmode="backward"),
                dict(count=6,label="6m",step="month",stepmode="backward"),
                dict(count=1,label="YTD",step="year",stepmode="todate"),
                dict(count=1,label="1y",step="year",stepmode="backward"),
                dict(step="all")])),
        rangeslider=dict(visible=True),type="date"))
    
    return fig_all_line_updated

@app.callback(
    Output('dist-chart', 'figure'),
    Input('year-range-slider', 'value'),
    Input('currency-radio', 'value')
)

def update_dist_plot(selected_years, selected_currency):
    # Filter clean_df based on selected years
    filtered_dist_df = clean_df[(clean_df['year'] >= selected_years[0]) & (clean_df['year'] <= selected_years[1])]
    
    if selected_currency == 'CZK_to_EUR':
        # Convert from CZK to EUR
        converted_hist = [filtered_dist_df['PX']]
        currency = 'EUR'
    elif selected_currency == 'EUR_to_CZK':
        # Convert from EUR to CZK
        converted_hist = [filtered_dist_df['PX'] * filtered_dist_df['EURCZK']]
        currency = 'CZK'
    else:
        converted_values = []

    #hist_df = [clean_df['PX']]
    #group_labels = ['PX Index'] 
    #colors = ['#3283FE']

    fig_dist_pdated = ff.create_distplot(converted_hist, group_labels, show_hist=False, colors=colors) 
    fig_dist_pdated.update_layout(title_text='PX Index Distribution',xaxis_title=currency,width=450,height=350,autosize=True,template="plotly_white", 
        legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1))

    return fig_dist_pdated

if __name__ == "__main__":
    app.run_server(debug=True, port=8083)
